In [90]:
import re
import string

from nltk.corpus import stopwords

import torch
from torchtext.datasets import AG_NEWS
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
from torch.utils.data import TensorDataset, DataLoader

In [85]:
# DataPipe 타입 >>> iterator 타입 형변환
train_iter = iter(AG_NEWS(split = 'train'))

In [86]:
# 데이터 확인 => (label, text), label : 1 ~ 4
next(train_iter)

(3,
 "Wall St. Bears Claw Back Into the Black (Reuters) Reuters - Short-sellers, Wall Street's dwindling\\band of ultra-cynics, are seeing green again.")

In [87]:
# 토커나이즈 생성
tokenizer = get_tokenizer('basic_english')

# 뉴스 학습 데이터 추출
train_iter = AG_NEWS(split = 'train')

In [88]:
stop_words_list = stopwords.words('english')
print('불용어 개수 :', len(stop_words_list))
print('불용어 10개 출력 :',stop_words_list[:10])

불용어 개수 : 179
불용어 10개 출력 : ['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're"]


In [91]:
# 토큰 제너레이터 함수 : 데이터 추출하여 토큰화
def yield_tokens(data_iter):
    for _, text in data_iter:
        # 라벨, 텍스트 => 텍스트 토큰화
        text_split = [word for word in text.split() if word not in stop_words_list + list(string.punctuation) + ['$']]
        text = ' '.join(text_split)
        text = re.sub('[^A-Za-z0-9가-힣]', ' ', text)  # 한글, 영어, 숫자만 남기고 특수문자 제거
        float_pattern = r"\b\d+\.\d+\b"  # 실수 제거
        text = re.sub(float_pattern, "", text)
        text = re.sub(r'\d', '', text)  # 정수 제거
        yield tokenizer(text)

In [92]:
for _, text in train_iter:
    print(_, text)
    print(type(text))
    break

3 Wall St. Bears Claw Back Into the Black (Reuters) Reuters - Short-sellers, Wall Street's dwindling\band of ultra-cynics, are seeing green again.
<class 'str'>


In [93]:
for text in yield_tokens(train_iter):
    print(_, text)
    print(type(text))
    break

3 ['wall', 'st', 'bears', 'claw', 'back', 'into', 'black', 'reuters', 'reuters', 'short', 'sellers', 'wall', 'street', 's', 'dwindling', 'band', 'ultra', 'cynics', 'seeing', 'green', 'again']
<class 'list'>


In [94]:
# 단어사전 생성
vocab = build_vocab_from_iterator(yield_tokens(train_iter), specials = ['<pad>', '<unk>'])

# <UNK> 인덱스 0으로 설정
vocab.set_default_index(vocab['<unk>'])

# <PAD> 인덱스 0으로 설정
vocab.set_default_index(vocab['<pad>'])

In [95]:
len(vocab)

61872

In [96]:
# 텍스트 => 정수 인코딩
text_pipeline = lambda x : vocab(tokenizer(x))

# 레이블 => 정수 인코딩
label_pipeline = lambda x : int(x) - 1

In [97]:
from torch.utils.data import DataLoader

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# 배치 크기만큼 데이터셋 반환 함수
def collate_batch(batch):
    label_list, text_list, offsets = [], [], [0]

    # 1개씩 뉴스기사, 라벨 추출해서 저장
    for (_label, _text) in batch:
        # 라벨 인코딩 후 저장
        label_list.append(label_pipeline(_label))

        # 텍스트 인코딩 후 저장
        processed_text = torch.tensor(text_pipeline(_text), dtype = torch.int64)
        text_list.append(processed_text)

        # 텍스트 offset 즉, 텍스트 크기/길이 저장
        offsets.append(processed_text.size(0))
    
    label_list = torch.tensor(label_list, dtype = torch.int64)
    offsets = torch.tensor(offsets[:-1]).cumsum(dim = 0)
    text_list = torch.cat(text_list)

    return label_list.to(device), text_list.to(device), offsets.to(device)

In [98]:
train_iter = AG_NEWS(split = 'train')
dataloader = DataLoader(train_iter,
                        batch_size = 8,
                        shuffle = True,
                        collate_fn = collate_batch)

In [99]:
num_class = len(set([label for (label, text) in train_iter]))
vocab_size = len(vocab)

print(f'num class : {num_class}   vocab size : {vocab_size}')

num class : 4   vocab size : 61872


In [100]:
for labels, texts, offsets in dataloader:
    print(labels, texts, offsets)
    break

tensor([1, 1, 1, 2, 3, 0, 3, 2]) tensor([ 8308,   217,   487, 23015,     0,     9,   666,  5164,   104,  3455,
          104,     3,  1256,   440,     3,    16,     0,   578,  5091,     3,
           15,    63,  2057,  4626,    15,    65,     0,   314,   188,  2190,
          173,     0,  1057, 23015,     0,    55,  2270,  1164,   152,  2097,
            0,  1738,     0,   232,  1136,     0,   314,  1726,  5494,   123,
          737,   440,   119,    23,  1375,  1738,   315,   232,    86,   979,
          104,  5455,   809, 20367,   483,   821,   115,   104,     8,  2717,
          612,     3,  6097,  6087,   429,  1289,   701,     0,     0,     0,
         4322, 29454,   358,   452,     0,     2,     0,  5396,   306,     0,
            6,     0,     0,   137,     0,     2, 12144, 29454, 14735,    63,
            8,   999,  1567,   104,    68,     3,   452,     0,     2,     0,
         2934,  5396,   217,   626,   245,     3,   306,   169,    25,     0,
         7831,    97,  1038,   

In [82]:
## 인코딩 : 문자 >>> 숫자로 변환
token_to_id = {voca : id for id, voca in enumerate(vocab)}

## 디코딩 : 숫자 >>> 문자로 변환
id_to_token = {id : voca for id, voca in enumerate(vocab)}

TypeError: __getitem__(): incompatible function arguments. The following argument types are supported:
    1. (self: torchtext._torchtext.Vocab, arg0: str) -> int

Invoked with: <torchtext._torchtext.Vocab object at 0x0000019978E6B870>, 0

In [ ]:
## 인코딩 : 문자 >>> 숫자로 변환
token_to_id = {voca : id for id, voca in enumerate(vocab)}

## 디코딩 : 숫자 >>> 문자로 변환
id_to_token = {id : voca for id, voca in enumerate(vocab)}

In [ ]:
# 리뷰에 문자를 정수로 변환 및 단어/어휘 사전에 없는 문자도 처리
UNK_ID = token_to_id.get('<UNK>')

train_ids = [[token_to_id.get(token, UNK_ID) for token in text] for text in train_tokens]
test_ids = [[token_to_id.get(token, UNK_ID) for token in text] for text in test_tokens]

In [ ]:
# 패딩 처리 함수

# - sentences : 토큰화된 문장 데이터
# - max_length : 최대 문장길이 즉, 1개 문장 구성 단어수
# - pad : 패딩 처리 시 추가될 문자 값
# - start : 패딩 시 처리 방향 [기:R 오른쪽 즉, 뒷부분 자르기/추가하기]
def pad_sequence(sentences, max_length, pad, start = 'R'):
    result = []
    for sen in sentences:
        sen = sen[:max_length] if start == 'R' else sen[:-1*max_length]
        padd_sen = sen + [pad]*(max_length-len(sen)) if start == 'R' else [pad]*(max_length-len(sen)) + sen
        result.append(padd_sen)
    
    return result

In [ ]:
# 학습용, 테스트용 데이터 패딩 처리
PAD_ID = token_to_id.get('<PAD>')
MAX_LENGTH = 32

train_ids = pad_sequence(train_ids, MAX_LENGTH, PAD_ID)
test_ids = pad_sequence(test_ids, MAX_LENGTH, PAD_ID)

In [ ]:
# 데이터셋 생성 : List를 Tensor로 변환

# 학습용 데이터셋
dataTS = torch.LongTensor(train_ids)
labelTS = torch.FloatTensor(trainDF.label.values)

print(f'dataTS => {dataTS.shape}, labelTS => {labelTS.shape}')

trainDS = TensorDataset(dataTS, labelTS)

In [ ]:
# 테스트용 데이터셋
dataTS = torch.LongTensor(test_ids)
labelTS = torch.FloatTensor(testDF.label.values)

print(f'dataTS => {dataTS.shape}, labelTS => {labelTS.shape}')

testDS = TensorDataset(dataTS, labelTS)

In [ ]:
# 데이터로더 생성
BATCH_SIZE = 32

trainDL = DataLoader(trainDS, BATCH_SIZE, shuffle = True)
testDL = DataLoader(testDS, BATCH_SIZE, shuffle = True)